# Instructions:
-	Your solution for this assignment need to be in a pdf format
-	For programming question, your code should be well documented, so anyone reading it can understand what's happening.


# Problem 1: Maximal Margin Classifier



| Obs| X1 | X2| Y  |
| ---|----|---|----|
| 1  | 3  | 4 | R  |
| 2  | 2  | 2 | R  |
| 3  | 4  | 4 | R  | 
| 4  | 1  | 4 | R  |
| 5  | 2  | 1 | B  |
| 6  | 4  | 3 | B  |
| 7  | 4  | 1 | B  |


### 1.  Here we explore the maximal margin classifier on a toy data set. 
#### Requisites: You should be able to easily solve this problem after reviewing Ch 9 Support Vector Machines of the ISLR book.

In the table above we are given n  = 7 observations in p  = 2 dimensions. For each observation, there is an associated class label. 

a) Plot the observations. 

b) Plot the optimal separating hyperplane.

c) Describe the classification rule for the maximal margin classifier. It should be something along the lines of "Classify to Red if ```β0 + β1X1 + β2X2 > 0```, and classify to Blue otherwise." Provide the values for β0 , β1 , and β2 

d) On your sketch, indicate the margin for the maximal margin hyperplane.

e) Indicate the support vectors for the maximal margin classifier.

f) Argue that a slight movement of the seventh observation would not affect the maximal margin hyperplane.

g) Plot an additional observation so that the two classes are no longer separable by a hyperplane.

### Problem 1 Solution

a) The observations are plotted in the figure below:

<img src="HW2_Plot1a.png">

b) The optimal separating hyperplane is indicated by the solid black line:

<img src="HW2_Plot1b.png">

c) First, we need to determine the equation for the hyperplane, which means determining the value of the coefficients $\beta_0$, $\beta_1$ and $\beta_2$

To do this, we need two points on the hyperplane. Two easy candidates are the midpoints between the blue and red points closest to the hyperplane, which are (2,1.5) and (4,3.5). Plugging these in to the hyperplane equation gives:

$\beta_0$ + 2$\beta_1$ + 1.5$\beta_2$ = 0

$\beta_0$ + 4$\beta_1$ + 3.5$\beta_2$ = 0

Multiplying the first equation by 2 and subtracting gives:

-$\beta_0$ + 0.5$\beta_2$ = 0 or $\beta_2$ = 2$\beta_0$

Plugging this into the first equation gives:

$\beta_0$ + 2$\beta_1$ + 3$\beta_0$ = 0

Solving for $\beta_1$ gives:

$\beta_1$ = -2$\beta_0$

Plugging everything in gives:

$\beta_0$ - 2$\beta_0$$X_1$ + 2$\beta_0$$X_2$ = 0

There are an infinite number of values for $\beta_0$ that satisfy this equation, so let's make it simple and choose $\beta_0$ = 1. With this, the final equation for the optimal hyperplane is:

1 - 2$X_1$ + 2$X_2$ = 0

so:

$\beta_0$ = 1

$\beta_1$ = -2

$\beta_2$ = 2

As a check, we can plug in the two points we started with to verify the equation holds:

For (2,1.5) we have

1 - 2(2) + 2(1.5) = 1 - 4 + 3 = 0

and for (4,3.5) we have

1 - 2(4) + 2(3.5) = 1 - 8 + 7 = 0

The classification rule is therefore:

"Classify to Red if 1 - 2$X_1$ + 2$X_2$ > 0 and classify to Blue if 1 - 2$X_1$ + 2$X_2$ < 0"

d) The margin for the maximal margin hyperplane is indicated by the two dotted lines on either side of the hyperplane in the figure below:

<img src="HW2_Plot1d.png">

e) The support vectors are the four points circled in the figure below:

<img src="HW2_Plot1e.png">

f) The seventh observation is indicated by the green square in the figure below. Because it is not a support vector, a slight movement would not affect the maximal margin hyperplane. Movement of this point would only affect the hyperplane if it moved so much that it became closer to the hyperplane than an existing support vector, in which case it would become a support vector itself and the equation of the maximal margin hyperplane would change.

<img src="HW2_Plot1f.png">

g) The addition of an 8th observation on the wrong side of the hyperplane, like the one indicated below with the arrow, would make it such that the two classes are no longer separable by a hyperplane.

<img src="HW2_Plot1g.png">

# Problem 2: Applied

### a) Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-all to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?

In [1]:
from sklearn.datasets import fetch_mldata
import numpy as np

#load the dataset 
mnist = fetch_mldata("MNIST original")
X = mnist["data"]
y = mnist["target"]

"""split it into a training set and a test set. We could use train_test_split() 
but people usually just take the first 60,000 instances for the training set,
and the last 10,000 instances for the test set (this makes it possible to compare your model's performance with others)
But feel free to play around with train/test split if you believe that would increase the accuracy.
"""
X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]
print(np.shape(X_train))
print(np.shape(X_test))

(60000, 784)
(10000, 784)


Since we're using a 'one-verus-all' approach, we can use the 'LinearSVC' estimator within the 'SVM' module of scikit-learn, which automatically performs 'one-versus-all' classification on multi-class data. 

We'll use a grid search based algorithm to find the 'best' values of the parameters for our classifier, which in this case are:
* C - a tuning parameter related to the sum of the error terms for misclassified observations
* tol - a tolerance for the stopping criteria
* loss - specifies the loss function to minimize

Note that the kernel is not a parameter we can change, since the 'LinearSVC' estimator uses a linear kernel.

To speed up the grid search portion of the parameter estimation, we'll randomly sample 10% of the training data.

In [2]:
# Import svm and grid search modules
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Define parameters for grid search
parameters = {'C':[1,10], 'tol':np.logspace(-6,-1,6), 'loss':('hinge','squared_hinge')}
# Get reduced set for tuning (10% of data)
frac = 0.1
num_obs = np.shape(X_train)[0]
tune_size = int(round(num_obs*frac))
# Uniform sample to get indices for tune set
np.random.seed(seed=32779)
tune_obs = np.random.randint(0,num_obs,tune_size)
X_tune = X_train[tune_obs,:]
y_tune = y_train[tune_obs]

In [3]:
# Perform grid search
gsCV = GridSearchCV(svm.LinearSVC(), parameters, verbose = 1)
gsCV.fit(X_tune,y_tune)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:  2.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'loss': ('hinge', 'squared_hinge'), 'C': [1, 10], 'tol': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

With the parameters tuned using the random subset, we can now create a model which uses all of the training data and the values of the best fit parameters.

In [4]:
bp = gsCV.best_params_
print('Best fit parameters from grid search:')
print(bp)

Best fit parameters from grid search:
{'loss': 'hinge', 'C': 1, 'tol': 0.0001}


In [5]:
# Create SVM model using best parameters from grid search
svmModel = svm.LinearSVC(C = bp['C'], tol = bp['tol'], loss = bp['loss'])
svmModel.fit(X_train,y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

With our trained model, we can now generate predicted values for the test data to estimate the accuracy of our classifier.

In [6]:
# Predict digits from test set
svmPredict = svmModel.predict(X_test)

We can generate a confusion matrix for the predicted results

In [7]:
from sklearn.metrics import confusion_matrix
# Generate confusion matrix for test results
cmatrix = confusion_matrix(y_test,svmPredict)
print('Confusion Matrix:')
print(cmatrix)

Confusion Matrix:
[[ 909    0   27    6    2   11   17    3    4    1]
 [   0 1123    6    0    0    1    2    1    2    0]
 [   0   20  949   10    5    5   10    8   23    2]
 [   1    7   84  841    2   28    2   22   12   11]
 [   0   10   21    1  894    2   10    5   10   29]
 [   9   12   23   67    8  685   21   15   43    9]
 [   6    7   37    0    6   26  876    0    0    0]
 [   0    9   49    1    4    2    1  934    2   26]
 [   4   76   70   40    8   42   22   11  683   18]
 [   3   16   34   14   60    9    0   68   28  777]]


Here, the diagonal terms represent correct classifications. For example, the first entry is the number of correct classifications of the digit '0', and the last entry is the number of correct classifications of the digit '9'. 

The off-diagonal terms represent the number of misclassifications. For example, the '27' in the first row, third column represents the number of times a '0' was mistakenly classified as a '2'. Overall, these misclassifications make sense, for example a '9' is most often misclassified as a '4' (60 times) or a '7' (68 times).

We can calculate the accuracy by summing the diagonal terms (which will give the total number of correct classifications), and divide by the number of test observations.

In [8]:
svmAccuracy = float(np.sum(np.diag(cmatrix)))/np.shape(y_test)[0] * 100
print('The accuracy of the SVM classifier on the test data set is %f%%' % svmAccuracy)

The accuracy of the SVM classifier on the test data set is 86.710000%


### b) Train a deep Multilayer Perceptron on the same dataset and compare the accuracy with the one above. Clearly explain your steps.

To do this, we're going to use the 'MLPClassifier' function from the 'neural_network' module. Since a deep MLP is specified, we'll make sure to use at least two hidden layers in our model. As before, there are several hyperparameters we can tune in order to maximize the accuracy of our classifier. For the multilayer perceptron, the parameters that will be tuned are:
* hidden_layer_sizes - specifies the number of hidden layers and the number of neurons in each layer
* activation - the activation function for the neurons in each hidden leyer

Also, to speed up the grid search portion of the parameter estimation, we'll once again use the same random sample of the training data (10%).

Note that the MLP is sensitive to feature scaling, but all features are already in the same units (pixels), so we don't need to adjust our data.

In [9]:
# Import neural_network module
from sklearn import neural_network
# Define parameters for grid search
# Set number of neurons in each hidden layer equal to the number of features
neurons = np.shape(X_train)[1]
# Include 2,3, and 4 hidden layers in grid search, along with logistic and relu activation functions
parameters = {'hidden_layer_sizes':[(neurons,neurons),(neurons,neurons,neurons),(neurons,neurons,neurons,neurons)], 
              'activation':('logistic','relu')}

In [10]:
# Perform grid search
gsCV = GridSearchCV(neural_network.MLPClassifier(), parameters, verbose = 1)
gsCV.fit(X_tune,y_tune)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 16.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'activation': ('logistic', 'relu'), 'hidden_layer_sizes': [(784, 784), (784, 784, 784), (784, 784, 784, 784)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

With the parameters tuned using the random subset, we can now create a model which uses all of the training data and the values of the best fit parameters. Note that using four hidden layers resulted in the best performance, along with the use of the relu (rectified linear) activation function.

In [11]:
bp = gsCV.best_params_
print('Best fit parameters from grid search:')
print(bp)

Best fit parameters from grid search:
{'activation': 'relu', 'hidden_layer_sizes': (784, 784, 784, 784)}


In [12]:
# Create MLP model using best parameters from grid search
mlpModel = neural_network.MLPClassifier(activation = bp['activation'],hidden_layer_sizes = bp['hidden_layer_sizes'])
mlpModel.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(784, 784, 784, 784), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

With our trained model, we can now generate predicted values for the test data to estimate the accuracy of our classifier.

In [13]:
# Predict digits from test set
mlpPredict = mlpModel.predict(X_test)

We can generate a confusion matrix for the predicted results

In [14]:
# Generate confusion matrix for test results
cmatrix = confusion_matrix(y_test,mlpPredict)
print('Confusion Matrix:')
print(cmatrix)

Confusion Matrix:
[[ 955    0    1    0    0    6   15    2    0    1]
 [   0 1130    2    0    0    0    1    0    2    0]
 [   2    6 1010    4    1    1    0    4    4    0]
 [   0    0    4  985    0   15    0    3    2    1]
 [   0    0    5    0  945    0   14    0    5   13]
 [   1    0    0    5    1  879    3    0    3    0]
 [   2    3    0    0    2    4  946    0    1    0]
 [   2   10   12    5    1    1    0  994    1    2]
 [   1    1    9    5    2   13    5    5  931    2]
 [   2    4    2    2   14   14    2    6    7  956]]


At a first glance, it appears the multilayer perceptron may have done a better job of classifying the digits, as the off-diagonal terms appear to be smaller than in the SVM case.

As before, we can calculate the accuracy by summing the diagonal terms (which will give the total number of correct classifications), and divide by the number of test observations.

In [15]:
# Calculate accuracy for MLP classifier
mlpAccuracy = float(np.sum(np.diag(cmatrix)))/np.shape(y_test)[0] * 100
print('The accuracy of the MLP classifier on the test data set is %f%%' % mlpAccuracy)

The accuracy of the MLP classifier on the test data set is 97.310000%


The accuracy of the MLP classifier is over 10% higher than that of the SVM, and nearly 100% at that. Although the MLP classifier took significantly longer in both grid search and training (during the grid search, the SVM classifier generated a model every 1.5 seconds, whereas the MLP classifier generated a model every 30 seconds), the tradeoff was worth it as we received a substantial increase in accuracy by switching to the deep multilayer perceptron model.